In [1]:
import pandas as pd
import sqlite3
import os

#### **Merging and Joining**
1. **Inner Join on Chinook Database**
   - Load the `chinook.db` database.
   - Perform an inner join between the `customers` and `invoices` tables on the `CustomerId` column.
   - Find the total number of invoices for each customer.

2. **Outer Join on Movie Data**
   - Load the `movie.csv` file.
   - Create two smaller DataFrames:
     - One with only `director_name` and `color`.
     - Another with `director_name` and `num_critic_for_reviews`.
   - Perform a left join and then a full outer join on `director_name`.
   - Count how many rows are in the resulting DataFrames for each join type.

---


In [2]:
# Part 1 Task 1
def load_chinook():
    try:
        path = r'D:\course_from_maab\lesson-19\data\chinook.db'
        with sqlite3.connect(path) as connection:
            customers = pd.read_sql(
                'SELECT * from customers',
                con = connection
            )
            invoices = pd.read_sql(
                'SELECT * from invoices',
                con = connection
            )
        return customers, invoices
    except sqlite3.OperationalError as e:
        print(f'Error occured {e}')
        return None
    except Exception as e:
        print(f'Unexpected error happen {e}')
        return None

In [3]:
def work_with_chinook(customers, invoices):
    inner_join = customers.merge(invoices, how = 'inner', on = 'CustomerId')    
    result = inner_join.groupby('CustomerId').agg({
        'InvoiceId': 'nunique',
        **{col: 'first' for col in inner_join.columns[0:13] if col != 'InvoiceId'}
    })
    result.rename(columns={'InvoiceId':'The total number of invoices'}, inplace=True)
    return inner_join, result

In [4]:
def load_movie():
    try:
        path = r'D:\course_from_maab\lesson-19\data\movie.csv'
        if not os.path.exists(path):
            print(f'File Not found at {path}')
            return None
        movie_data = pd.read_csv(path)
        return movie_data
    except Exception as e:
        print(f'Unexpected error happen {e}')
        return None

In [5]:
def work_with_movies(movie_data):
    dataFrame_colors = movie_data.loc[:, ['director_name', 'color']]
    dataFrame_critics = movie_data.loc[:, ['director_name', 'num_critic_for_reviews']]
    left_merged = dataFrame_colors.merge(dataFrame_critics, how = 'left', on = 'director_name')
    outer_merged = dataFrame_colors.merge(dataFrame_critics, how = 'outer', on = 'director_name')
    return left_merged.shape[0], outer_merged.shape[0]


#### **Grouping and Aggregating**
1. **Grouped Aggregations on Titanic**
   - Group passengers by `Pclass` and calculate the following:
     - Average age.
     - Total fare.
     - Count of passengers.
   - Save the results to a new DataFrame.

2. **Multi-level Grouping on Movie Data**
   - Group the movies by `color` and `director_name`.
   - Find:
     - Total `num_critic_for_reviews` for each group.
     - Average `duration` for each group.

3. **Nested Grouping on Flights**
   - Group flights by `Year` and `Month` and calculate:
     - Total number of flights.
     - Average arrival delay (`ArrDelay`).
     - Maximum departure delay (`DepDelay`).

---


In [6]:
def load_titanic():
    try:
        path = r'D:\course_from_maab\lesson-19\data\titanic.xlsx'
        titanic_data = pd.read_excel(path, sheet_name=0)
        return titanic_data
    except Exception as e:
        print(f'Unexpected error happen {e}')
        return None

In [7]:
def grouping_titanic(titanic_data):
    titanic_statistics = titanic_data.groupby(by='Pclass').agg({
        'Age':'mean',
        'Fare':'sum',
        'PassengerId':'nunique' }).reset_index()
    titanic_statistics.rename(columns = {'PassengerId':'Number of passengers'}, inplace = True)
    return titanic_statistics

In [8]:
def grouping_movies(movie_data):
    movie_statistics = movie_data.groupby(by=['director_name', 'color']).agg({
        'num_critic_for_reviews':'sum',
        'duration':'mean'}).reset_index()
    return movie_statistics

In [9]:
def load_flights():
    try:
        path = r'D:\course_from_maab\lesson-19\data\flights'
        if not os.path.exists(path):
            print(f'File not found in {path}')
            return None
        flights_data = pd.read_parquet(path)
        return flights_data

    except Exception as e:
        print(f'Error occured {e}')
        return None

In [10]:
def grouping_flights(flights_data):
    flights_data.loc[:, 'Flights'] = pd.to_numeric(flights_data.loc[:, 'Flights'], errors='coerce')
    flights_data.loc[:, 'ArrDelay'] = pd.to_numeric(flights_data.loc[:, 'ArrDelay'], errors='coerce')
    flights_data.loc[:, 'DepDelay'] = pd.to_numeric(flights_data.loc[:, 'DepDelay'], errors='coerce')
    statistics_flights = flights_data.groupby(by=['Year' ,'Month']).agg({
        'Flights':'sum',
        'ArrDelay':'sum',
        'DepDelay':'sum' 
    })
    return statistics_flights

---

#### **Applying Functions**
1. **Apply a Custom Function on Titanic**
   - Write a function to classify passengers as `Child` (age < 18) or `Adult`.
   - Use `apply` to create a new column, `Age_Group`, with these values.

2. **Normalize Employee Salaries**
   - Load the `employee.csv` file.
   - Normalize the salaries within each department.

3. **Custom Function on Movies**
   - Write a function that returns `Short`, `Medium`, or `Long` based on the duration of a movie:
     - `Short`: Less than 60 minutes.
     - `Medium`: Between 60 and 120 minutes.
     - `Long`: More than 120 minutes.
   - Apply this function to classify movies in the `movie.csv` dataset.

---


In [11]:
def custom_function_on_titanic(titanic_data):
    def classify_func(age):
        if age < 18:
            return 'Child'
        else:
            return 'Adult'
    titanic_data['Age_Group'] = titanic_data['Age'].apply(classify_func)
    return titanic_data

In [12]:
def load_employee():
    try:
        path = r'D:\course_from_maab\lesson-19\data\employee.csv'
        if not os.path.exists(path):
            print(f'File not found in {path}')
            return None
        employee_data = pd.read_csv(path)
        return employee_data

    except Exception as e:
        print(f'Error occured {e}')
        return None

In [13]:
def reshape_employee(employee_data):
    employee_data['normalized_by_salaries'] = employee_data.groupby(by='DEPARTMENT')['BASE_SALARY'].transform(lambda x: (x - x.mean())/x.std())
    return employee_data

In [14]:
def custom_functions_on_movie(movie_data):
    def classify_movies(duration):
        if duration < 60:
            return 'Short'
        elif duration > 120:
            return 'Long'
        else:
            return 'Medium'
    movie_data['Classify_by_duration'] = movie_data['duration'].apply(classify_movies)
    return movie_data

#### **Using `pipe`**
1. **Pipeline on Titanic**
   - Create a pipeline to:
     - Filter passengers who survived (`Survived == 1`).
     - Fill missing `Age` values with the mean.
     - Create a new column, `Fare_Per_Age`, by dividing `Fare` by `Age`.

2. **Pipeline on Flights**
   - Create a pipeline to:
     - Filter flights with a departure delay greater than 30 minutes.
     - Add a column `Delay_Per_Hour` by dividing the delay by the scheduled flight duration.

In [15]:
def work_with_titanic(titanic_data):
    def filter_survived(titanic_data):
        return titanic_data.loc[titanic_data['Survived'] == 1].copy()
    def fill_missing_ages(titanic_data):
        titanic_data['Age'] = titanic_data['Age'].fillna(titanic_data['Age'].mean())
        return titanic_data
    def fare_per_age(titanic_data):
        titanic_data['Fare_Per_Age'] = titanic_data['Fare']/titanic_data['Age']
        return titanic_data
    titanic_data = titanic_data.pipe(filter_survived).pipe(fill_missing_ages).pipe(fare_per_age)
    return titanic_data

In [16]:
def work_with_flights(flights_data):
    
    def filter_delay(flights_data):
        flights_data['DepDelay'] = pd.to_numeric(flights_data['DepDelay'])
        return flights_data[(flights_data['DepDelay'] > 30) | (flights_data['DepDelay'] < -30)].copy()

    def delay_per_hour(flights_data):
        flights_data['DepDelay'] = pd.to_numeric(flights_data['DepDelay'])
        flights_data['AirTime'] = pd.to_numeric(flights_data['AirTime'])
        flights_data['Delay_Per_Hour'] = flights_data['DepDelay'] / flights_data['AirTime']
        return flights_data

    flights_data = flights_data.pipe(filter_delay).pipe(delay_per_hour)

    return flights_data


In [17]:
def display_part1(movie_data):
    print('Part1 Answers:')
    customers, invoices = load_chinook()
    print(work_with_chinook(customers, invoices))
    print(work_with_movies(movie_data))


In [18]:
def display_part2(titanic_data, movie_data, flights_data):
    print('Part2 Answers:')
    print(grouping_titanic(titanic_data))
    print(grouping_movies(movie_data))
    print(grouping_flights(flights_data))

In [19]:
def display_part3(titanic_data, movie_data):
    print('Part3 Answers:')
    employee_data = load_employee()
    print(work_with_titanic(titanic_data))
    print(custom_functions_on_movie(movie_data))
    print(reshape_employee(employee_data))


In [20]:
def display_part4(titanic_data, flights_data):
    print('Part4 Answers:')
    print(work_with_titanic(titanic_data))
    print(work_with_flights(flights_data))

In [21]:
def main():
    movie_data = load_movie()
    titanic_data = load_titanic()
    flights_data = load_flights()
    display_part1(movie_data)
    display_part2(titanic_data, movie_data, flights_data)
    display_part3(titanic_data, movie_data)
    display_part4(titanic_data, flights_data)

In [ ]:
if __name__ == '__main__':
    main()